In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.signal import convolve
from ipywidgets import Checkbox, VBox, Button, Output
print("Libraries added successfully!")

Libraries added successfully!


In [2]:
def ask_errors(k,M,nsamp,EbN0_db):
    
    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
    SNR = 10 ** (SNR_db * 0.1)
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    P_x = (L*L-1) / 3
    Measured_x = np.sum(x*x)/len(x)

    y = []
    for i in range(len(x)):
        for j in range(nsamp):
            y.append(x[i])
    y = np.array(y)

    noise = np.random.normal(0, np.sqrt(Measured_x/SNR),len(y))
    y_noisy = y + noise

    y = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp,1))
    z = np.matmul(y, matched)
    z = z / nsamp
    l = np.arange(-L+1,L,2)

    z = z[:,0]
    for i in range(len(z)):
        differences = np.abs(l-z[i]) # Πίνακας με τις διαφορές του σήματος από τα επίπεδα
        m = min(differences)
        [index], = np.where(differences == m)
        z[i] = l[index]
    
    errors = 0
    for i in range(len(z)):
        if x[i] != z[i]:
            errors += 1
    
    return errors


In [3]:
def generate_histogram(k):
    M = 40000
    L = 2 ** k

    x = 2 * np.floor(L * np.random.rand(M)) - L + 1
    bins = np.arange(-L, L + 2, 2)
    A = np.arange(-(L - 1), L, 2)
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 4))
    ax.hist(x, bins=bins, edgecolor='white', color='#1F77B4')  # Use ax.hist for compatibility with widgets
    ax.set_xticks(A)
    ax.set_xlabel("Integers")
    ax.set_ylabel("Frequency")
    ax.set_title("Histogram of array x elements")
    plt.show()

k_input = widgets.IntText(
    value=3,  # default value
    description='k:',
    continuous_update=False
)

ui = widgets.VBox([k_input])
out = widgets.interactive_output(generate_histogram, {'k': k_input})

# Display the UI components
display(ui, out)

Output()

In [4]:
def generate_histogram(k, EbN0_db):
    M = 60000
    nsamp = 16

    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
    SNR = 10 ** (SNR_db * 0.1)
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    P_x = (L**2 - 1) / 3
    Measured_x = np.sum(x**2)/len(x)

    y = np.repeat(x, nsamp)

    noise = np.random.normal(0, np.sqrt(Measured_x/SNR), len(y))
    y_noisy = y + noise

    y_reshaped = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp, 1))
    z = np.matmul(y_reshaped, matched) / nsamp

    A = np.arange(-(L-1), L, 2)  # Adjusted to use inside plotting
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 4))
    ax.hist(z, bins=200,  edgecolor='white', color='#1F77B4')
    ax.set_xticks(A)
    ax.set_xlabel("Integers")
    ax.set_ylabel("Frequency")
    ax.legend(["Eb/N0 = " + str(EbN0_db)])
    ax.set_title('Histogram of the Noisy Signal')
    plt.show()

k_input = widgets.IntText(
    value=4,  # default value
    description='k:',
    continuous_update=False
)

EbN0_db_slider = widgets.FloatSlider(
    value=10,  # default value
    min=0,    # minimum value
    max=20,   # maximum value
    step=0.1,   # step size
    description='Eb/N0 (dB):',
    continuous_update=False
)

ui = widgets.VBox([k_input, EbN0_db_slider])

out = widgets.interactive_output(generate_histogram, {'k': k_input, 'EbN0_db': EbN0_db_slider})

# Display the UI components
display(ui, out)

Output()

In [5]:
def ask_errors(k,M,nsamp,EbN0_db):
    
    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
    SNR = 10 ** (SNR_db * 0.1)
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    P_x = (L*L-1) / 3
    Measured_x = np.sum(x*x)/len(x)

    y = []
    for i in range(len(x)):
        for j in range(nsamp):
            y.append(x[i])
    y = np.array(y)

    noise = np.random.normal(0, np.sqrt(Measured_x/SNR),len(y))
    y_noisy = y + noise

    y = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp,1))
    z = np.matmul(y, matched)
    z = z / nsamp
    l = np.arange(-L+1,L,2)

    z = z[:,0]
    for i in range(len(z)):
        differences = np.abs(l-z[i]) # Πίνακας με τις διαφορές του σήματος από τα επίπεδα
        m = min(differences)
        [index], = np.where(differences == m)
        z[i] = l[index]
    
    errors = 0
    for i in range(len(z)):
        if x[i] != z[i]:
            errors += 1
    
    return errors


M = 20000
nsamp = 16
EbN0_db = np.arange(0, 18, 1)
EbN0 = 10**(EbN0_db / 10)

# Create checkboxes for each modulation level
checkbox_4qam = Checkbox(value=True, description='4-QAM')
checkbox_8qam = Checkbox(value=False, description='8-QAM')
checkbox_16qam = Checkbox(value=False, description='16-QAM')

plot_output = Output()

def plot_selected_modulations(btn=None):
    with plot_output:
        clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        
        # Define colors for each modulation level
        colors = {
            '4-QAM': ('red', 'tomato'),
            '8-QAM': ('green', 'limegreen'),
            '16-QAM': ('blue', 'dodgerblue')
        }
        
        for i, (k, checkbox) in enumerate(zip([2, 3, 4], [checkbox_4qam, checkbox_8qam, checkbox_16qam])):
            if checkbox.value:
                L = 2**k
                modulation_name = f'{L}-QAM'
                ber = np.zeros(len(EbN0_db))
                for index, eb_n0 in enumerate(EbN0_db):
                    # Simulate experimental BER using a placeholder function
                    ber[index] = ask_errors(k, M, nsamp, index) / M / np.log2(L)
                
                # Plot experimental BER as points without connecting lines, using specific colors
                plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}', color=colors[modulation_name][0])
                
                # Theoretical BER for comparison, with distinct line color
                ber_theoretical = (((L-1)/L) * erfc(np.sqrt(EbN0 * (3 * np.log2(L)) / (L**2 - 1)))) / k
                plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}', color=colors[modulation_name][1])

        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("BER")
        plt.legend()
        plt.show()

# Button to update the plot
plot_button = Button(description="Plot")
plot_button.on_click(plot_selected_modulations)

# Initially plot with default settings
plot_selected_modulations()

# Displaying the widgets and the output container
display(VBox([checkbox_4qam, checkbox_8qam, checkbox_16qam, plot_button]), plot_output)

Output()